In [9]:
import cv2
import numpy as np

"""Empty function for trackbar callbacks. Does nothing."""
def nothing(x):
    pass
    
# Initialize video capture object
cap = cv2.VideoCapture(0)

# Create the trackbar window
cv2.namedWindow("name")

# Create trackbars for HSV color range selection
cv2.createTrackbar("upper_hue","name",55,180,nothing)
cv2.createTrackbar("upper_saturation","name",255, 255, nothing)
cv2.createTrackbar("upper_value","name",255, 255, nothing)
cv2.createTrackbar("lower_hue","name",17,180, nothing)
cv2.createTrackbar("lower_saturation","name",62, 255, nothing)
cv2.createTrackbar("lower_value","name",0, 255, nothing)

# Read the first frame to initialize
while True:
    cv2.waitKey(1000)
    r,init_frame =cap.read()
    if(r):
        break
        
        
while True:
    # Read a frame from the video capture
    r,frame = cap.read()
    
    # Convert the frame from BGR to HSV color space
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    # Get the current trackbar positions for HSV ranges
    upper_hue = cv2.getTrackbarPos("upper_hue", "name")
    upper_saturation = cv2.getTrackbarPos("upper_saturation", "name")
    upper_value = cv2.getTrackbarPos("upper_value", "name")
    lower_value = cv2.getTrackbarPos("lower_value","name")
    lower_hue = cv2.getTrackbarPos("lower_hue","name")
    lower_saturation = cv2.getTrackbarPos("lower_saturation","name")

    # Define the kernel for morphological operations
    kernal =np.ones((3,3),np.uint16)
    
    # Create NumPy arrays for upper and lower HSV bounds
    upper_hsv = np.array([upper_hue,upper_saturation,upper_value])
    lower_hsv = np.array([lower_hue,lower_saturation,lower_value])
    
    # Create a mask using inRange to identify pixels within the color range
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    
    # Apply median blur to reduce noise in the mask
    mask =cv2.medianBlur(mask,3)
    
    # Invert the mask to isolate the background (areas outside the color range)
    mask_inv = 255 - mask  # black
    
    # Dilate the mask to enhance the object boundaries (optional)
    mask = cv2.dilate(mask,kernal,5)

    # Extract individual color channels from the frame
    b = frame[:,:,0]
    g = frame[:,:,1]
    r = frame[:,:,2]
    
    # Apply bitwise AND with the inverted mask to make the background black
    b = cv2.bitwise_and(mask_inv, b)
    g = cv2.bitwise_and(mask_inv, g)
    r = cv2.bitwise_and(mask_inv, r)
    
    # Merge the modified channels back into a color image (black background area)
    black_sheet_area = cv2.merge((b,g,r))
    

    # Extract individual color channels from the initial frame
    b = init_frame[:,:,0]
    g = init_frame[:,:,1]
    r = init_frame[:,:,2]
    
    # Apply bitwise AND with the mask to extract the color-matching object
    b = cv2.bitwise_and(mask , b)
    g = cv2.bitwise_and(mask , g)
    r = cv2.bitwise_and(mask , r)
    
    # Merge the modified channels back into a color image (object area)
    sheet_area = cv2.merge((b,g,r))
  
    
    # Combine the object and background areas using bitwise OR
    result = cv2.bitwise_or(sheet_area,black_sheet_area)
    cv2.imshow("final result",cv2.resize(result,(700,370)))
    cv2.imshow("org",cv2.resize(frame,(700,370)))
    
    if(cv2.waitKey(3) == ord('q')):
        break
        
cv2.destroyAllWindows()
cap.release()